In [1]:
import tensorflow as tf
import numpy as np
tf.__version__

2022-12-22 19:03:17.443465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 19:03:18.150989: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2022-12-22 19:03:18.151122: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2022-12-22 19:03:18.151129: W tensorflow/compiler/tf2tensorrt/utils/py_utils

'2.11.0'

## Customizing Models and Training Algorithm

### Custom Loss Functions

In [2]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# Use vectorized implementation for better performance.
# Only use tensorfow operation for graph features.

In [3]:
# Construct model
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[8]),
    tf.keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal"),
    tf.keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal"),
    tf.keras.layers.Dense(1),
])

2022-12-22 19:03:19.126135: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-22 19:03:19.131352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-22 19:03:19.131725: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-22 19:03:19.132296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [4]:
# Use custom loss to compile model
model.compile(loss=huber_fn, optimizer="nadam")

### Saving and Loading Models That Contain Custom Components

In [5]:
model_folder = "/home/adminvbdi/Desktop/Hands-On-Machine-Learning-Tensorflow-and-Keras/Chapter 10: Introduction to ANN with Keras/my_keras_model.h5"
model = tf.keras.models.load_model(model_folder, custom_objects={"huber_fn": huber_fn})

In [6]:
# Create a function that creates a configured loss function
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer="nadam")

In [7]:
# Unfortunately, when you save the model, the threshold will not be saved. This means
# that you will have to specify the threshold value when loading the model (note that
# the name to use is "huber_fn" , which is the name of the function you gave Keras, not
# the name of the function that created it):
model = tf.keras.models.load_model(model_folder, 
                            custom_objects={"huber_fn": create_huber(2.0)})

In [10]:
# Another way
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [11]:
model.compile(loss=HuberLoss(2.0), optimizer="nadam")

In [12]:
# save model
# When you save the model, the threshold will be saved along with it
model.save(model_folder)

In [13]:
# when you load the model, you just need to map the class name to the class itself:
model = tf.keras.models.load_model(model_folder,
                                custom_objects={"HuberLoss": HuberLoss})

### Custom Activation Functions, Initializers, Regularizers, and Constraints

In [14]:
# custom activation function
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

In [15]:
# custom initializer
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

In [16]:
# custom regularizer
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))


In [17]:
# custom constraint
def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [18]:
layer = tf.keras.layers.Dense(30, activation=my_softplus,
                            kernel_initializer=my_glorot_initializer,
                            kernel_regularizer=my_l1_regularizer,
                            kernel_constraint=my_positive_weights)

In [19]:
# If a function has hyperparameters that need to be saved along with 
# the model, then you will want to subclass the appropriate class.
# Much like we did for custom loss

class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

> **Note that, you must implement the `call()` method for losses, layer and activation functions, and models, or `__call__()` for regularizers, initializers, and constraints.**

### Custom Metrics

> Losses and metrics are conceptually not the same thing: losses (e.g., cross entropy)
are used by Gradient Descent to train a model, so they must be differentiable (at least
where they are evaluated), and their gradients should not be 0 everywhere. Plus, it’s
OK if they are not easily interpretable by humans. In contrast, metrics (e.g., accuracy)
are used to evaluate a model: they must be more easily interpretable, and they can be
non-differentiable or have 0 gradients everywhere.

> That said, in most cases, defining a custom metric function is exactly the same as
defining a custom loss function. In fact, we could even use the Huber loss function we
created earlier as a metric; 6 it would work just fine (and persistence would also work
the same way, in this case only saving the name of the function, "huber_fn" ):

In [20]:
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

# Wrong way

> We must create a subclass of tf.keras.metrics.Metric and implement the following.

In [21]:
# Create a function that creates a configured loss function
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn


class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

### Custom Layers